In [1]:
import datetime
import pickle
import numpy as np
import pandas as pd
import glob
import os.path
from pyproj import Transformer
from tqdm import tqdm
import os
from pathlib import Path
import geopandas as gpd

In [20]:
city = 'NYC'
df = pd.read_csv(
    f"../data/raw/dataset_TSMC2014_{city}.csv.zip",
    parse_dates=[-1],
    date_format="%a %b %d %H:%M:%S %z %Y",
)
df = df.rename(
    columns={
        "utcTimestamp": "datetime",
        "longitude": "lon",
        "latitude": "lat",
        "userId": "user",
    }
)
df = df.sort_values(by=['user', 'datetime'])

tmp = []
for t_idx, tdf in df.groupby("user"):
    traj_start = tdf["datetime"].iloc[0]
    tdf["sub_traj"] = (tdf["datetime"] - traj_start) // datetime.timedelta(hours=6)
    tmp.append(tdf)
df = pd.concat(tmp)
df = df.sort_values(by=["user", "datetime"])
df["t_idx"] = df.groupby(["user", "sub_traj"]).ngroup()
# df.to_pickle(f"../data/processed/foursquare_{city}.pkl")

In [44]:
tmp = []
for t_idx, tdf in df.groupby("user"):
    traj_start = tdf["datetime"].iloc[0]
    tdf["sub_traj"] = (tdf["datetime"] - traj_start) // datetime.timedelta(hours=6)
    tmp.append(tdf)
df = pd.concat(tmp)
df = df.sort_values(by=["user", "datetime"])
df["t_idx"] = df.groupby(["user", "sub_traj"]).ngroup()

In [49]:
most_active_users = df.groupby('user').size().nlargest(209).index

In [45]:
dff = df.groupby("t_idx").filter(lambda x: len(x) > 1)

In [50]:
df[df['user'].isin(most_active_users)].groupby('t_idx').size().describe()

count    44558.000000
mean         2.200391
std          1.965806
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max         64.000000
dtype: float64

In [22]:
df.groupby('t_idx').size().describe()

count    135118.000000
mean          1.683181
std           1.455645
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max          71.000000
dtype: float64

In [11]:
for city in ["NYC", 'TKY'] :
    df = pd.read_csv(
        f"../data/raw/dataset_TSMC2014_{city}.csv.zip",
        parse_dates=[-1],
        date_format="%a %b %d %H:%M:%S %z %Y",
    )
    df = df.rename(
        columns={
            "utcTimestamp": "datetime",
            "longitude": "lon",
            "latitude": "lat",
            "userId": "user",
        }
    )
    df = df.sort_values(by=['user', 'datetime'])
    tmp = []
    for user, dfu in df.groupby('user'):
        dfu['timediff'] = dfu['datetime'] - dfu['datetime'].shift(1)
        tmp.append(dfu)
    df = pd.concat(tmp)

    df['timediff'] = df['timediff'].fillna(datetime.timedelta(seconds=0))
    # Identify the rows where the time difference exceeds 6 hours
    df['new_trajectory'] = (df['timediff'] > datetime.timedelta(hours=8)).astype(int)
    # Create a cumulative sum to mark distinct subtrajectories
    df['t_idx'] = df.groupby('user')['new_trajectory'].cumsum()
    df['t_idx'] = df.groupby(['user', 't_idx']).ngroup()

    tmp = []
    for t_idx, tdf in df.groupby("t_idx"):
        traj_start = tdf["datetime"].iloc[0]
        tdf["sub_traj"] = (tdf["datetime"] - traj_start) // datetime.timedelta(hours=24)
        tmp.append(tdf)
    df = pd.concat(tmp)
    df = df.sort_values(by=["user", "datetime"])
    df["t_idx"] = df.groupby(["t_idx", "sub_traj"]).ngroup()
    df.to_pickle(f"../data/processed/foursquare_{city}.pkl")

KeyboardInterrupt: 

In [50]:
df.groupby('t_idx').size().describe()

count    222601.000000
mean          2.577271
std           2.675923
min           1.000000
25%           1.000000
50%           2.000000
75%           3.000000
max          78.000000
dtype: float64

In [ ]:
gow = pd.read_csv("../data/raw/loc-gowalla_totalCheckins.txt.gz", names=['user', 'datetime', 'lat', 'lon', 'location'], parse_dates=[1], sep="\t")

gow = gow.sort_values(['user', 'datetime'])
tmp = []
for user, dfu in gow.groupby('user'):
    dfu['timediff'] = dfu['datetime'] - dfu['datetime'].shift()
    tmp.append(dfu)

gow = pd.concat(tmp)